# UMADock With Agent

## Set-up

In [1]:
!pip install --upgrade --quiet  langchain-huggingface
!pip install langchain_core --quiet
! pip -q install langchain_community
!pip install -q langgraph
!pip install -q transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 21.1 MB/s eta 0:00:00


In [12]:
import torch
import re
from typing import Annotated, TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import SystemMessage, trim_messages, AIMessage, HumanMessage, ToolCall

from langchain_huggingface.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import chain

from gradio_client import Client

In [7]:
hf = HuggingFacePipeline.from_model_id(
    #model_id= "swiss-ai/Apertus-8B-Instruct-2509",
    model_id= "microsoft/Phi-4-mini-instruct",
    task="text-generation",
    pipeline_kwargs = {"max_new_tokens": 500, "temperature": 0.4})

chat_model = ChatHuggingFace(llm=hf)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0


## Define Agent
- replace URL in client with gradio link (see UMADock with Gradio notebook)

In [28]:
client = Client('https://132075be3162a9a635.gradio.live')

Loaded as API: https://132075be3162a9a635.gradio.live/ ✔


In [ ]:
test = client.predict(query_smiles, query_protein)

In [29]:
class State(TypedDict):
  '''
    The state of the agent.
  '''
  messages: Annotated[list, add_messages]
  query_smiles: str
  query_protein: str
  props_string: str

def first_node(state: State) -> State:
  '''
    The first node of the agent. This node receives the input and asks the LLM
    to determine which is the best tool to use to answer the QUERY TASK.

      Input: the initial prompt from the user. should contain only one of more of the following:

             smiles: the smiles string, protein: the target protein

             the value should be separated from the name by a ':' and each field should
             be separated from the previous one by a ','.

             All of these values are saved to the state

      Output: the tool choice
  '''
  query_smiles = None
  query_protein = None
  props_string = ""
  state["props_string"] = props_string

  raw_input = state["messages"][-1].content
  parts = raw_input.split(',')
  for part in parts:
    if 'smiles' in part:
      query_smiles = part.split(':')[1]
      state["query_smiles"] = query_smiles
    if 'protein' in part:
      query_protein = part.split(':')[1]
      state["query_protein"] = query_protein

  print(f"query_smiles: {query_smiles}")
  print(f"query_protein: {query_protein}")

  props_string += client.predict(query_smiles, query_protein)
  state["props_string"] = props_string

  return state

def parser_node(state: State) -> State:
  '''
    This is the third node in the agent. It receives the output from the tool,
    puts it into a prompt as CONTEXT, and asks the LLM to answer the original
    query.

      Input: the output from the tool.
      Output: the answer to the original query.
  '''
  props_string = state["props_string"]
  query_smiles = state["query_smiles"]
  query_protein = state["query_protein"]

  prompt = f'Using the CONTEXT below, discuss the results from the docking calculation, \
Wwhere the molecule represented by the SMILES string was docked in the PROTEIN. \n \
SMILES: {query_smiles}.\n \
PROTEIN: {query_protein}.\n \
CONTEXT: {props_string}.\n '

  res = chat_model.invoke(prompt)
  return {"messages": res}

def pretty_print(answer):
  final = str(answer['messages'][-1]).split('<|assistant|>')[-1].split('#')[0].strip("n").strip('\\').strip('n').strip('\\')
  for i in range(0,len(final),100):
    print(final[i:i+100])

def print_short(answer):
  for i in range(0,len(answer),100):
    print(answer[i:i+100])

In [30]:
builder = StateGraph(State)
builder.add_node("first_node", first_node)
builder.add_node("parser_node", parser_node)

builder.add_edge(START, "first_node")
builder.add_edge("first_node", "parser_node")

builder.add_edge("parser_node", END)

graph = builder.compile()

## Run Agent

In [31]:
input = {
    "messages": [
        HumanMessage('smiles: c1ccc(F)cc1, protein:DRD2')
    ]
}

In [32]:
cs = []
for c in graph.stream(input): #, stream_mode='updates'):
  m = re.findall(r'[a-z]+\_node', str(c))
  if len(m) != 0:
    reply = c[str(m[0])]['messages']
    if 'assistant' in str(reply):
      reply = str(reply).split("<|assistant|>")[-1].split('#')[0].strip()
      print_short(reply)
      print('===================================================')
    elif "HumanMessage" in str(reply):
      reply = str(reply).split("content='")[1].split(", additional_k")[0]
  cs.append(c)

query_smiles:  c1ccc(F)cc1
query_protein: DRD2
The docking calculation results indicate that the molecule represented by the SMILES string "c1ccc(F
)cc1" (which is 1-fluoro-2,5-dimethylbenzene) has been docked into the DRD2 protein. The lowest elec
tronic binding energy observed was from conformer 1, with a binding energy of -13.520 kcal/mol. This
 suggests that in pose 1, the 1-fluoro-2,5-dimethylbenzene molecule interacts with the DRD2 protein 
in a manner that results in a stable and energetically favorable binding configuration. The negative
 value of the binding energy indicates that the interaction is exothermic, meaning that the binding 
of the molecule to the protein releases energy, which is a favorable outcome in drug design and mole
cular docking studies. The specific details of the interaction, such as the binding site on the DRD2
 protein and the nature of the interactions (e.g., hydrogen bonds, hydrophobic interactions, etc.), 
would provide further insights into the pote

In [33]:
input = {
    "messages": [
        HumanMessage('smiles: O=C(O)[C@@](NN)(Cc1cc(O)c(O)cc1)C, protein:DRD2')
    ]
}

In [34]:
cs = []
for c in graph.stream(input): #, stream_mode='updates'):
  m = re.findall(r'[a-z]+\_node', str(c))
  if len(m) != 0:
    reply = c[str(m[0])]['messages']
    if 'assistant' in str(reply):
      reply = str(reply).split("<|assistant|>")[-1].split('#')[0].strip()
      print_short(reply)
      print('===================================================')
    elif "HumanMessage" in str(reply):
      reply = str(reply).split("content='")[1].split(", additional_k")[0]
  cs.append(c)

query_smiles:  O=C(O)[C@@](NN)(Cc1cc(O)c(O)cc1)C
query_protein: DRD2
The docking calculation results indicate that the molecule represented by the SMILES string O=C(O)[C
@@](NN)(Cc1cc(O)c(O)cc1)C was docked into the DRD2 protein. The specific conformer and pose that res
ulted in the lowest electronic binding energy were identified as conformer 8 and pose 0, respectivel
y. The binding energy for this pose was calculated to be -12.034 kcal/mol. This negative value sugge
sts a favorable interaction between the molecule and the DRD2 protein, indicating that the molecule 
may be a potential ligand or inhibitor that could bind effectively to the DRD2 protein. The docking 
results provide valuable insights into the potential binding affinity and interaction of the molecul
e with the target protein, which can be further investigated for drug discovery and development purp
oses.' additional_kwargs={} response_metadata={} id='run--f33efdb2-9d6c-4725-ad60-8b34fc767f20-0'
